<a href="https://colab.research.google.com/github/cemgundogan/DLAI_carbon_aware_computing_for_genAI_devs/blob/main/L2%20Explore%20Carbon%20Intensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 2: Explore Carbon Intensity on the Grid

* In this classroom, the libraries have been already installed for you.
* If you would like to run this code on your own machine, make sure to get your Electricity Maps API from their [website](https://www.electricitymaps.com/free-tier-api).

#### Load the Electricity Maps API for this notebook

In [ ]:
from helper import load_env
load_env()

### Electricity Map's live carbon intensity  endpoint

* Set location coordinates.

Remember, you can get the coordinates from [Google Maps](https://www.google.com/maps).

In [ ]:
# Create a dictionary with the coordinates
coordinates = {
    "lat":34.00906474557528,
    "lon": -118.4984580927553
}

* Request from the endpoint.

In [ ]:
# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=34.00906474557528&lon=-118.4984580927553


* Prepare the get request.

In [ ]:
import requests
import helper

In [ ]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [ ]:
# This byte format is more compact
request.content
type(request.content)

bytes

> Note: This byte format is more compact and often not human-readable. Additionally, it is not possible to access the fields within this byte data as we would access with key-value pairs in a dictionary. This is why we use json.loads()

In [ ]:
import json

In [ ]:
json.loads(request.content)

{'zone': 'US-CAL-LDWP',
 'carbonIntensity': 700,
 'datetime': '2024-06-26T17:00:00.000Z',
 'updatedAt': '2024-06-26T16:49:48.834Z',
 'createdAt': '2024-06-23T17:46:14.865Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Use the live power breakdown endpoint.

In [ ]:
# Build the url
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

In [ ]:
print(url)

https://api.electricitymap.org/v3/power-breakdown/latest?lat=34.00906474557528&lon=-118.4984580927553


In [ ]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

In [ ]:
power_breakdown = json.loads(request.content)

# Print the content
power_breakdown

{'zone': 'US-CAL-LDWP',
 'datetime': '2024-06-26T17:00:00.000Z',
 'updatedAt': '2024-06-26T16:49:48.834Z',
 'createdAt': '2024-06-23T17:46:14.865Z',
 'powerConsumptionBreakdown': {'nuclear': 0,
  'geothermal': 0,
  'biomass': 0,
  'coal': 0,
  'wind': 0,
  'solar': 0,
  'hydro': 0,
  'gas': 0,
  'oil': 0,
  'unknown': 191,
  'hydro discharge': 0,
  'battery discharge': 0},
 'powerProductionBreakdown': {'nuclear': None,
  'geothermal': None,
  'biomass': None,
  'coal': None,
  'wind': None,
  'solar': None,
  'hydro': None,
  'gas': None,
  'oil': None,
  'unknown': 191,
  'hydro discharge': None,
  'battery discharge': None},
 'powerImportBreakdown': {},
 'powerExportBreakdown': {},
 'fossilFreePercentage': 0,
 'renewablePercentage': 0,
 'powerConsumptionTotal': 191,
 'powerProductionTotal': 191,
 'powerImportTotal': None,
 'powerExportTotal': None,
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

* Print some specific values.

In [ ]:
power_breakdown['renewablePercentage']

0

In [ ]:
power_breakdown['fossilFreePercentage']

0

In [ ]:
# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

{'nuclear': 0,
 'geothermal': 0,
 'biomass': 0,
 'coal': 0,
 'wind': 0,
 'solar': 0,
 'hydro': 0,
 'gas': 0,
 'oil': 0,
 'unknown': 191,
 'hydro discharge': 0,
 'battery discharge': 0}

* Do some math to understand better the values above.

In [ ]:
import numpy as np

In [ ]:
total_consumption = power_breakdown['powerConsumptionTotal']

In [ ]:
total_consumption

191

In [ ]:
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': 0.0,
 'geothermal': 0.0,
 'biomass': 0.0,
 'coal': 0.0,
 'wind': 0.0,
 'solar': 0.0,
 'hydro': 0.0,
 'gas': 0.0,
 'oil': 0.0,
 'unknown': 100.0,
 'hydro discharge': 0.0,
 'battery discharge': 0.0}

#### Helper function for the power_stats

In [ ]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100)
            for k, v
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }

    return intensity, breakdown_abridged

In [ ]:
# Coordinates from a landmark in Taiwan, shown by the instructor in the explanation
intensity, breakdown = power_stats(
    lat=25.0356575108668,
    lon=121.52010809479746)

In [ ]:
intensity

{'zone': 'TW',
 'carbonIntensity': 582,
 'datetime': '2024-06-26T17:00:00.000Z',
 'updatedAt': '2024-06-26T16:50:08.753Z',
 'createdAt': '2024-06-23T17:46:14.865Z',
 'emissionFactorType': 'lifecycle',
 'isEstimated': True,
 'estimationMethod': 'TIME_SLICER_AVERAGE'}

In [ ]:
breakdown

{'renewablePercentage': 2,
 'fossilFreePercentage': 8,
 'powerConsumptionBreakdown': {'nuclear': 1799,
  'geothermal': 2,
  'biomass': 24,
  'coal': 11270,
  'wind': 65,
  'solar': 0,
  'hydro': 533,
  'gas': 16308,
  'oil': 377,
  'unknown': 1267,
  'hydro discharge': 0,
  'battery discharge': 0},
 'consumption_percent': {'nuclear': 6.0,
  'geothermal': 0.0,
  'biomass': 0.0,
  'coal': 36.0,
  'wind': 0.0,
  'solar': 0.0,
  'hydro': 2.0,
  'gas': 52.0,
  'oil': 1.0,
  'unknown': 4.0,
  'hydro discharge': 0.0,
  'battery discharge': 0.0}}

### Do it yourself!
* Get coordinates from a location you want to retrieve the information we got before!

In [ ]:
from helper import load_env
load_env()

# Create a dictionary with the coordinates
coordinates = {
    "lat":51.5188583894350968,
    "lon": -0.08145885941370525
}

# Build the url
url= f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

# Print the endpoint
print("Endpoint: " + str(url))

Endpoint: https://api.electricitymap.org/v3/carbon-intensity/latest?lat=51.518858389435096&lon=-0.08145885941370525


In [ ]:
import requests
import helper

request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

# This byte format is more compact
request.content
type(request.content)

import json

json.loads(request.content)

# Build the url
url = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"

print(url)

https://api.electricitymap.org/v3/power-breakdown/latest?lat=51.518858389435096&lon=-0.08145885941370525


In [ ]:
request = requests.get(
    url,
    headers={"auth-token": helper.load_emaps_api_key()})

power_breakdown = json.loads(request.content)

# Print the content
power_breakdown
power_breakdown['renewablePercentage']
power_breakdown['fossilFreePercentage']

# Power Consumption Breakdown in MegaWatts
power_breakdown['powerConsumptionBreakdown']

import numpy as np

total_consumption = power_breakdown['powerConsumptionTotal']
total_consumption
consumption_percent = {
    k: np.round((v/total_consumption) * 100)
    for k,v
    in power_breakdown['powerConsumptionBreakdown'].items()}
consumption_percent

{'nuclear': 27.0,
 'geothermal': 0.0,
 'biomass': 6.0,
 'coal': 0.0,
 'wind': 7.0,
 'solar': 10.0,
 'hydro': 4.0,
 'gas': 41.0,
 'oil': 0.0,
 'unknown': 3.0,
 'hydro discharge': 2.0,
 'battery discharge': 0.0}

In [ ]:
import helper, requests, json, numpy as np
def power_stats(lat,lon, api_key=helper.load_emaps_api_key()):
    coordinates = { "lat": lat, "lon": lon }

    url_intensity = f"https://api.electricitymap.org/v3/carbon-intensity/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_intensity = requests.get(url_intensity, headers={"auth-token": api_key})
    intensity = json.loads(request_intensity.content)

    url_breakdown = f"https://api.electricitymap.org/v3/power-breakdown/latest?lat={coordinates['lat']}&lon={coordinates['lon']}"
    request_breakdown = requests.get(url_breakdown, headers={"auth-token": api_key})
    breakdown = json.loads(request_breakdown.content)

    breakdown_abridged = {
        'renewablePercentage': breakdown['renewablePercentage'],
        'fossilFreePercentage': breakdown['fossilFreePercentage'],
        'powerConsumptionBreakdown': breakdown['powerConsumptionBreakdown'],
        'consumption_percent': {
            k: np.round((v/breakdown['powerConsumptionTotal']) * 100)
            for k, v
            in breakdown['powerConsumptionBreakdown'].items()
        },
    }

    return intensity, breakdown_abridged

In [ ]:
# Coordinates from London Liverpool Street Station, entered by the student Arif Cem Gundogan
intensity, breakdown = power_stats(
    lat=51.5188583894350968,
    lon=-0.08145885941370525)